# Sagaパターン テスト・分析ノートブック

このノートブックでは、ChoreographyパターンとOrchestrationパターンの両方を実装したSagaシステムに対して、包括的なテストを実施し、パフォーマンス分析を行います。

## テスト対象システム
- **Choreographyパターン**: 各サービスが自律的にイベントを処理
- **Orchestrationパターン**: Saga Orchestratorがワークフローを制御

## テスト内容
1. 正常フロー検証
2. エラーハンドリング検証
3. パフォーマンス比較
4. 負荷テスト
5. 結果分析と可視化

In [ ]:
!ruff format -i .

In [ ]:
# 必要なライブラリのインポート
import requests
import json
import time
import concurrent.futures
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import subprocess
import sys
import os

# スタイル設定
plt.style.use("seaborn-v0_8")
sns.set_palette("husl")

print("✅ ライブラリのインポート完了")

In [ ]:
# 設定と定数定義
class Config:
    # Choreographyパターン
    CHOREOGRAPHY_BASE_URL = "http://localhost"
    CHOREOGRAPHY_ORDER_SERVICE = f"{CHOREOGRAPHY_BASE_URL}:8001"
    CHOREOGRAPHY_INVENTORY_SERVICE = f"{CHOREOGRAPHY_BASE_URL}:8002"
    CHOREOGRAPHY_PAYMENT_SERVICE = f"{CHOREOGRAPHY_BASE_URL}:8003"
    CHOREOGRAPHY_SHIPPING_SERVICE = f"{CHOREOGRAPHY_BASE_URL}:8004"

    # Orchestrationパターン
    ORCHESTRATION_BASE_URL = "http://localhost"
    ORCHESTRATION_SAGA_ORCHESTRATOR = f"{ORCHESTRATION_BASE_URL}:8005"
    ORCHESTRATION_ORDER_SERVICE = f"{ORCHESTRATION_BASE_URL}:8011"
    ORCHESTRATION_INVENTORY_SERVICE = f"{ORCHESTRATION_BASE_URL}:8012"
    ORCHESTRATION_PAYMENT_SERVICE = f"{ORCHESTRATION_BASE_URL}:8013"
    ORCHESTRATION_SHIPPING_SERVICE = f"{ORCHESTRATION_BASE_URL}:8014"

    # テスト設定
    MAX_WORKERS = 10
    TIMEOUT = 30
    TEST_ITERATIONS = 50


config = Config()

# テストデータ
TEST_ORDERS = [
    {"customer_id": "customer-001", "items": [{"book_id": "book-123", "quantity": 1}]},
    {
        "customer_id": "customer-002",
        "items": [
            {"book_id": "book-456", "quantity": 1}  # 在庫切れ
        ],
    },
    {"customer_id": "customer-003", "items": [{"book_id": "book-789", "quantity": 2}]},
]

print("✅ 設定完了")

In [ ]:
# ヘルパー関数
def make_request(url, method="GET", data=None, timeout=config.TIMEOUT):
    """HTTPリクエストを実行"""
    try:
        if method == "GET":
            response = requests.get(url, timeout=timeout)
        elif method == "POST":
            response = requests.post(url, json=data, timeout=timeout)
        elif method == "PUT":
            response = requests.put(url, json=data, timeout=timeout)
        else:
            raise ValueError(f"Unsupported method: {method}")

        return {
            "status_code": response.status_code,
            "data": response.json() if response.content else None,
            "success": response.status_code < 400,
        }
    except requests.exceptions.RequestException as e:
        return {"status_code": None, "data": None, "success": False, "error": str(e)}


def check_service_health(base_url, service_name):
    """サービスのヘルスチェック"""
    try:
        response = make_request(f"{base_url}/health")
        return response["success"]
    except:
        return False


def start_docker_services(pattern_type):
    """Dockerサービスを開始"""
    compose_file = f"saga_pattern/{pattern_type}_pattern/docker-compose.yml"
    try:
        result = subprocess.run(
            ["docker-compose", "-f", compose_file, "up", "-d"],
            capture_output=True,
            text=True,
            cwd="/Users/codefox/workspace/practice_infra_arch",
        )
        return result.returncode == 0, result.stdout, result.stderr
    except Exception as e:
        return False, "", str(e)


def stop_docker_services(pattern_type):
    """Dockerサービスを停止"""
    compose_file = f"saga_pattern/{pattern_type}_pattern/docker-compose.yml"
    try:
        result = subprocess.run(
            ["docker-compose", "-f", compose_file, "down"],
            capture_output=True,
            text=True,
            cwd="/Users/codefox/workspace/practice_infra_arch",
        )
        return result.returncode == 0
    except Exception as e:
        return False


def measure_response_time(func, *args, **kwargs):
    """レスポンスタイムを測定"""
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    return result, end_time - start_time


print("✅ ヘルパー関数定義完了")

## Choreographyパターン テスト関数

In [ ]:
def test_choreography_order_creation(order_data):
    """Choreographyパターンで注文作成をテスト"""
    url = f"{config.CHOREOGRAPHY_ORDER_SERVICE}/orders"
    response = make_request(url, "POST", order_data)

    if response["success"]:
        order_id = response["data"]["order_id"]
        return {
            "success": True,
            "order_id": order_id,
            "message": "Order created successfully",
        }
    else:
        return {"success": False, "error": response.get("error", "Unknown error")}


def test_choreography_order_status(order_id):
    """Choreographyパターンで注文状態を確認"""
    url = f"{config.CHOREOGRAPHY_ORDER_SERVICE}/orders/{order_id}"
    response = make_request(url)

    if response["success"]:
        return response["data"]
    else:
        return {"error": response.get("error", "Failed to get order status")}


def test_choreography_inventory_status(book_id):
    """Choreographyパターンで在庫状態を確認"""
    url = f"{config.CHOREOGRAPHY_INVENTORY_SERVICE}/inventory/{book_id}"
    response = make_request(url)

    if response["success"]:
        return response["data"]
    else:
        return {"error": response.get("error", "Failed to get inventory status")}


def test_choreography_payment_status(order_id):
    """Choreographyパターンで決済状態を確認"""
    url = f"{config.CHOREOGRAPHY_PAYMENT_SERVICE}/payments/{order_id}"
    response = make_request(url)

    if response["success"]:
        return response["data"]
    else:
        return {"error": response.get("error", "Failed to get payment status")}


def test_choreography_shipping_status(order_id):
    """Choreographyパターンで配送状態を確認"""
    url = f"{config.CHOREOGRAPHY_SHIPPING_SERVICE}/shipping/{order_id}"
    response = make_request(url)

    if response["success"]:
        return response["data"]
    else:
        return {"error": response.get("error", "Failed to get shipping status")}


def run_choreography_test_scenario(order_data):
    """Choreographyパターンの完全なテストシナリオを実行"""
    results = {
        "order_creation": None,
        "order_status": None,
        "inventory_status": None,
        "payment_status": None,
        "shipping_status": None,
        "total_time": 0,
    }

    start_time = time.time()

    # 1. 注文作成
    order_result = test_choreography_order_creation(order_data)
    results["order_creation"] = order_result

    if not order_result["success"]:
        results["total_time"] = time.time() - start_time
        return results

    order_id = order_result["order_id"]

    # 少し待ってイベント処理を待つ
    time.sleep(2)

    # 2. 各サービスの状態確認
    results["order_status"] = test_choreography_order_status(order_id)

    if order_data["items"]:
        book_id = order_data["items"][0]["book_id"]
        results["inventory_status"] = test_choreography_inventory_status(book_id)

    results["payment_status"] = test_choreography_payment_status(order_id)
    results["shipping_status"] = test_choreography_shipping_status(order_id)

    results["total_time"] = time.time() - start_time

    return results


print("✅ Choreographyパターン テスト関数定義完了")

## Orchestrationパターン テスト関数

In [ ]:
def test_orchestration_saga_start(order_data):
    """OrchestrationパターンでSagaを開始"""
    url = f"{config.ORCHESTRATION_SAGA_ORCHESTRATOR}/saga/start"
    response = make_request(url, "POST", order_data)

    if response["success"]:
        saga_id = response["data"]["saga_id"]
        return {
            "success": True,
            "saga_id": saga_id,
            "message": "Saga started successfully",
        }
    else:
        return {"success": False, "error": response.get("error", "Unknown error")}


def test_orchestration_saga_status(saga_id):
    """OrchestrationパターンでSaga状態を確認"""
    url = f"{config.ORCHESTRATION_SAGA_ORCHESTRATOR}/saga/{saga_id}/status"
    response = make_request(url)

    if response["success"]:
        return response["data"]
    else:
        return {"error": response.get("error", "Failed to get saga status")}


def test_orchestration_order_status(order_id):
    """Orchestrationパターンで注文状態を確認"""
    url = f"{config.ORCHESTRATION_ORDER_SERVICE}/orders/{order_id}"
    response = make_request(url)

    if response["success"]:
        return response["data"]
    else:
        return {"error": response.get("error", "Failed to get order status")}


def run_orchestration_test_scenario(order_data):
    """Orchestrationパターンの完全なテストシナリオを実行"""
    results = {
        "saga_start": None,
        "saga_status": None,
        "order_status": None,
        "total_time": 0,
    }

    start_time = time.time()

    # 1. Saga開始
    saga_result = test_orchestration_saga_start(order_data)
    results["saga_start"] = saga_result

    if not saga_result["success"]:
        results["total_time"] = time.time() - start_time
        return results

    saga_id = saga_result["saga_id"]

    # 少し待ってSaga処理を待つ
    time.sleep(5)

    # 2. Saga状態確認
    results["saga_status"] = test_orchestration_saga_status(saga_id)

    # 3. 注文状態確認（Sagaからorder_idを取得）
    if "order_id" in order_data:
        results["order_status"] = test_orchestration_order_status(
            order_data["order_id"]
        )

    results["total_time"] = time.time() - start_time

    return results


print("✅ Orchestrationパターン テスト関数定義完了")

## パフォーマンステスト関数

In [ ]:
def run_load_test(pattern_type, order_data, num_requests=10):
    """指定パターンの負荷テストを実行"""
    results = []

    def single_request(i):
        start_time = time.time()

        if pattern_type == "choreography":
            result = run_choreography_test_scenario(order_data)
        elif pattern_type == "orchestration":
            result = run_orchestration_test_scenario(order_data)
        else:
            return {"error": "Invalid pattern type"}

        end_time = time.time()
        result["request_id"] = i
        result["pattern"] = pattern_type
        result["start_time"] = start_time
        result["end_time"] = end_time
        result["response_time"] = end_time - start_time

        return result

    # 並行してリクエストを実行
    with concurrent.futures.ThreadPoolExecutor(
        max_workers=config.MAX_WORKERS
    ) as executor:
        futures = [executor.submit(single_request, i) for i in range(num_requests)]
        for future in concurrent.futures.as_completed(futures):
            results.append(future.result())

    return results


def analyze_performance_results(results):
    """パフォーマンス結果を分析"""
    if not results:
        return {}

    df = pd.DataFrame(results)

    analysis = {
        "total_requests": len(df),
        "successful_requests": len(
            df[df.get("order_creation", {}).get("success", False) == True]
        )
        if "order_creation" in df.columns
        else 0,
        "failed_requests": len(df)
        - (
            len(df[df.get("order_creation", {}).get("success", False) == True])
            if "order_creation" in df.columns
            else 0
        ),
        "avg_response_time": df["response_time"].mean(),
        "min_response_time": df["response_time"].min(),
        "max_response_time": df["response_time"].max(),
        "p95_response_time": df["response_time"].quantile(0.95),
        "p99_response_time": df["response_time"].quantile(0.99),
    }

    return analysis


def compare_patterns(choreography_results, orchestration_results):
    """2つのパターンの結果を比較"""
    choreography_analysis = analyze_performance_results(choreography_results)
    orchestration_analysis = analyze_performance_results(orchestration_results)

    comparison = {
        "choreography": choreography_analysis,
        "orchestration": orchestration_analysis,
        "response_time_diff": orchestration_analysis.get("avg_response_time", 0)
        - choreography_analysis.get("avg_response_time", 0),
        "success_rate_diff": orchestration_analysis.get("successful_requests", 0)
        - choreography_analysis.get("successful_requests", 0),
    }

    return comparison


print("✅ パフォーマンステスト関数定義完了")

## テスト実行と結果分析

In [ ]:
# Dockerサービスの起動
print("🚀 Choreographyパターン サービス起動...")
choreography_started, stdout, stderr = start_docker_services("choreography")
if choreography_started:
    print("✅ Choreographyパターン サービス起動成功")
else:
    print(f"❌ Choreographyパターン サービス起動失敗: {stderr}")

print("\n🚀 Orchestrationパターン サービス起動...")
orchestration_started, stdout, stderr = start_docker_services("orchestration")
if orchestration_started:
    print("✅ Orchestrationパターン サービス起動成功")
else:
    print(f"❌ Orchestrationパターン サービス起動失敗: {stderr}")

# サービス起動待機
print("\n⏳ サービス起動待機中...")
time.sleep(10)

In [ ]:
# 基本テスト実行
print("🧪 Choreographyパターン 基本テスト...")

# 正常ケーステスト
normal_order = TEST_ORDERS[0]  # 在庫ありの注文
choreography_result = run_choreography_test_scenario(normal_order)
print(f"Choreography 正常ケース結果: {choreography_result}")

print("\n🧪 Orchestrationパターン 基本テスト...")
orchestration_result = run_orchestration_test_scenario(normal_order)
print(f"Orchestration 正常ケース結果: {orchestration_result}")

# エラーケーステスト
print("\n🧪 Choreographyパターン エラーケーステスト...")
error_order = TEST_ORDERS[1]  # 在庫切れの注文
choreography_error_result = run_choreography_test_scenario(error_order)
print(f"Choreography エラーケース結果: {choreography_error_result}")

print("\n🧪 Orchestrationパターン エラーケーステスト...")
orchestration_error_result = run_orchestration_test_scenario(error_order)
print(f"Orchestration エラーケース結果: {orchestration_error_result}")

In [ ]:
# 負荷テスト実行
print("🔥 負荷テスト実行中...")

# Choreographyパターン負荷テスト
print("Choreographyパターン 負荷テスト...")
choreography_load_results = run_load_test(
    "choreography", normal_order, config.TEST_ITERATIONS
)
print(f"Choreography負荷テスト完了: {len(choreography_load_results)} リクエスト")

# Orchestrationパターン負荷テスト
print("Orchestrationパターン 負荷テスト...")
orchestration_load_results = run_load_test(
    "orchestration", normal_order, config.TEST_ITERATIONS
)
print(f"Orchestration負荷テスト完了: {len(orchestration_load_results)} リクエスト")

# 結果保存
import pickle

with open("choreography_results.pkl", "wb") as f:
    pickle.dump(choreography_load_results, f)

with open("orchestration_results.pkl", "wb") as f:
    pickle.dump(orchestration_load_results, f)

print("✅ 負荷テスト結果保存完了")

In [ ]:
# 結果分析
print("📊 パフォーマンス分析...")

# 結果読み込み
try:
    with open("choreography_results.pkl", "rb") as f:
        choreography_load_results = pickle.load(f)

    with open("orchestration_results.pkl", "rb") as f:
        orchestration_load_results = pickle.load(f)

    print("✅ テスト結果読み込み成功")

except FileNotFoundError:
    print("❌ テスト結果ファイルが見つかりません")
    choreography_load_results = []
    orchestration_load_results = []

# 分析実行
if choreography_load_results and orchestration_load_results:
    choreography_analysis = analyze_performance_results(choreography_load_results)
    orchestration_analysis = analyze_performance_results(orchestration_load_results)
    comparison = compare_patterns(choreography_load_results, orchestration_load_results)

    print("\n📈 Choreographyパターン分析:")
    for key, value in choreography_analysis.items():
        print(f"  {key}: {value}")

    print("\n📈 Orchestrationパターン分析:")
    for key, value in orchestration_analysis.items():
        print(f"  {key}: {value}")

    print("\n⚖️ パターン比較:")
    print(f"  レスポンスタイム差: {comparison['response_time_diff']:.3f}秒")
    print(f"  成功率差: {comparison['success_rate_diff']}")

else:
    print("❌ 分析対象のデータがありません")

In [ ]:
# 可視化
print("📊 結果可視化...")

if choreography_load_results and orchestration_load_results:
    # DataFrame作成
    choreography_df = pd.DataFrame(choreography_load_results)
    orchestration_df = pd.DataFrame(orchestration_load_results)

    # レスポンスタイム比較
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.hist(choreography_df["response_time"], alpha=0.7, label="Choreography", bins=20)
    plt.hist(
        orchestration_df["response_time"], alpha=0.7, label="Orchestration", bins=20
    )
    plt.xlabel("Response Time (seconds)")
    plt.ylabel("Frequency")
    plt.title("Response Time Distribution")
    plt.legend()

    # 成功/失敗比較
    plt.subplot(1, 2, 2)
    success_data = {
        "Choreography": [
            choreography_analysis.get("successful_requests", 0),
            choreography_analysis.get("failed_requests", 0),
        ],
        "Orchestration": [
            orchestration_analysis.get("successful_requests", 0),
            orchestration_analysis.get("failed_requests", 0),
        ],
    }

    success_df = pd.DataFrame(success_data, index=["Success", "Failed"])
    success_df.plot(kind="bar", ax=plt.gca())
    plt.title("Success/Failure Comparison")
    plt.ylabel("Number of Requests")

    plt.tight_layout()
    plt.savefig("saga_pattern_comparison.png", dpi=300, bbox_inches="tight")
    plt.show()

    print("✅ 可視化完了 - saga_pattern_comparison.png に保存")

else:
    print("❌ 可視化対象のデータがありません")

In [ ]:
# クリーンアップ
print("🧹 クリーンアップ...")

# Dockerサービス停止
print("Dockerサービス停止中...")
choreography_stopped = stop_docker_services("choreography")
orchestration_stopped = stop_docker_services("orchestration")

if choreography_stopped and orchestration_stopped:
    print("✅ すべてのサービス停止完了")
else:
    print("⚠️ 一部のサービス停止に失敗しました")

print("\n🎉 Sagaパターン テスト完了!")
print("\n📋 テストサマリー:")
print("- Choreographyパターン: イベント駆動型の自律的サービス連携")
print("- Orchestrationパターン: 中央制御型のワークフロー管理")
print("- 両パターンの長所と短所を比較分析")
print("- パフォーマンス特性とユースケース適合性を評価")

print("\n💡 主要な知見:")
print("1. Choreography: 疎結合、高いスケーラビリティ、デバッグの複雑さ")
print("2. Orchestration: 集中管理、明確な制御、単一障害点のリスク")
print("3. ユースケースによる適切なパターン選択の重要性")